C﻿hapter 1: Using supervised classification with random forest to detect seagrass and unvegetated flats.

In [ ]:
import xlrd
import statsmodels.api as sm
## gdal
from osgeo import gdal, gdal_array
import numpy as np
from numpy.linalg import inv
from scipy.stats.distributions import chi2
from itertools import chain
import matplotlib.pyplot as plt
from osgeo import ogr
## AI
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, accuracy_score, f1_score, precision_score, recall_score


In [ ]:
raster = gdal.Open('/content/gdrive/My Drive/Reflectance/Rrs20220224/Rrs20220224_cl.tif') ##Please merge your individual band to one Gtiff if you are using Sentinel-2 data

vector = ogr.Open('/content/gdrive/My Drive/Supervised_classification/classification_reproject_summer1.shp')

lyr = vector.GetLayer()

geot = raster.GetGeoTransform()

proj = raster.GetProjectionRef()

vec_raster = gdal.GetDriverByName('GTiff')

chn_ras = vec_raster.Create('/content/gdrive/My Drive/Supervised_classification/supervised_classification_ras_Summer.tif',raster.RasterXSize,raster.RasterYSize,1,gdal.GDT_Byte)

chn_ras.SetGeoTransform(geot)

chn_ras.SetProjection(proj)

gdal.RasterizeLayer(chn_ras, [1], lyr, options=['ATTRIBUTE=id']) # the field name should correspond to the classificaiton that you would like to use for supervised classification

chn_ras.GetRasterBand(1).SetNoDataValue(0)

chn_ras = None*## This process can also be done in ArcGIS or QGIS.*

# import the raster data we have just converted from shapefile\

vector_raster = gdal.Open('/content/gdrive/My Drive/Supervised_classification/supervised_classification_ras_Summer.tif')

vec = vector_raster.GetRasterBand(1).ReadAsArray()

#Create a container to save the result

width = raster.RasterYSize; lenth = raster.RasterXSize; number = raster.RasterCount

container = np.ones((width, lenth, number),gdal_array.GDALTypeCodeToNumericTypeCode(raster.GetRasterBand(1).DataType))

for i in range(number):
  container[:,:,i] = raster.GetRasterBand(i+1).ReadAsArray().astype(np.float32)

fit_X  = container[vec>0,:]

fit_y  = vec[vec>0]

In [ ]:
fit_X_train, fit_X_test, fit_y_train, fit_y_test = train_test_split(fit_X,fit_y,test_size = 0.7)

In [ ]:
RandomForest = RandomForestClassifier(n_estimators=280,min_samples_split=4,min_samples_leaf=4,max_depth=90)`
randomforest = RandomForest.fit(fit_X_train,fit_y_train)
y_predict = randomforest.predict(fit_X_test)


In [ ]:
Final_container = (container.shape[0]*container.shape[1],container.shape[2])

Final_container_re = container[:,:,:4].reshape(Final_container)

supervised_classification = randomforest.predict(Final_container_re)

supervised_classification = supervised_classification.reshape(container[:,:,0].shape) # reshape the results`

plt.imshow(supervised_classification) # show the results`

plt.colorbar()

In [ ]:
import joblib

joblib.dump(randomforest,"/content/gdrive/My Drive/Supervised_classification/model_summer.joblib")



raster = gdal.Open('/content/gdrive/My Drive/Reflectance/Rrs20211231/Rrs20220224_cl.tif') ##This tif file is used for georeference`

geot = raster.GetGeoTransform()

proj = raster.GetProjectionRef()

filename = '/content/gdrive/My Drive/Supervised_classification/sc20220221.tif'

xsize = np.shape(supervised_classification)[1]

ysize = np.shape(supervised_classification)[0]

driver = gdal.GetDriverByName('GTiff')

dataset = driver.Create(filename, xsize, ysize, bands = 1, eType = gdal.GDT_UInt16)

dataset.GetRasterBand(1).WriteArray(supervised_classification)

dataset.SetGeoTransform(geot)

dataset.SetProjection(proj)

dataset.FlushCache()

dataset=None